In [1]:
%matplotlib inline

In [2]:
from pathlib import Path
import re
import matplotlib.pyplot as plt
from matplotlib import pyplot
import eelbrain
import mne
#import trftools
from pprint import pprint
import numpy as np

import emd  # Ensure you ran: pip install EMD-signal
from scipy.signal import hilbert

import csv
import pandas as pd

/Users/neuroling/anaconda3/envs/eelbrain/lib/python3.11/site-packages/eelbrain/mne_fixes/_interpolation.py:13: FutureWarning: mne.io.pick.pick_types is deprecated will be removed in 1.6, use documented public API instead. If no appropriate public API exists, please open an issue on GitHub.
  from mne.io.pick import pick_types, pick_channels
/Users/neuroling/anaconda3/envs/eelbrain/lib/python3.11/site-packages/eelbrain/mne_fixes/_interpolation.py:13: FutureWarning: mne.io.pick.pick_channels is deprecated will be removed in 1.6, use documented public API instead. If no appropriate public API exists, please open an issue on GitHub.
  from mne.io.pick import pick_types, pick_channels


In [9]:
## ESLs ##
## Import the raw EEG data of ESLs(Alice)

STIMULI = [str(i) for i in range(1, 13)]
DATA_ROOT = Path("/Users/neuroling/Downloads/DINGHSIN_Results/Alice_Experiments_Results")
#DATA_ROOT = Path("/Volumes/Neurolang_1/Master Program/New_Thesis_topic/Experiments_Results")  #Path("~").expanduser() / 'Data' / 'Alice'
PREDICTOR_audio_DIR = DATA_ROOT / 'TRFs_pridictors/audio_predictors'
PREDICTOR_word_DIR = DATA_ROOT / 'TRFs_pridictors/word_predictors'
EEG_DIR = DATA_ROOT / 'EEG_ESLs' / 'Alice_ESL_ICAed_fif'
ESL_SUBJECTS = [path.name for path in EEG_DIR.iterdir() if re.match(r'n_2_S\d*', path.name)]  #S01_alice-raw.fif
# Define a target directory for TRF estimates and make sure the directory is created
TRF_DIR = DATA_ROOT / 'TRFs_ESLs'
TRF_DIR.mkdir(exist_ok=True)
print(ESL_SUBJECTS)
print(len(ESL_SUBJECTS))  # 26

DST = TRF_DIR / 'ESLs_figures'
DST.mkdir(exist_ok=True)

wOnset_DIR = DATA_ROOT / 'EEG_ESLs' / 'Alice_ESLs_wOnset_raw_epochs'
wOnset_DIR .mkdir(exist_ok=True)

['n_2_S030_ICAed_raw.fif', 'n_2_S027_ICAed_raw.fif', 'n_2_S023_ICAed_raw.fif', 'n_2_S034_ICAed_raw.fif', 'n_2_S024_ICAed_raw.fif', 'n_2_S019_ICAed_raw.fif', 'n_2_S020_ICAed_raw.fif', 'n_2_S013_ICAed_raw.fif', 'n_2_S017_ICAed_raw.fif', 'n_2_S039_ICAed_raw.fif', 'n_2_S010_ICAed_raw.fif', 'n_2_S029_ICAed_raw.fif', 'n_2_S015_ICAed_raw.fif', 'n_2_S028_ICAed_raw.fif', 'n_2_S011_ICAed_raw.fif', 'n_2_S038_ICAed_raw.fif', 'n_2_S016_ICAed_raw.fif', 'n_2_S012_ICAed_raw.fif', 'n_2_S021_ICAed_raw.fif', 'n_2_S036_ICAed_raw.fif', 'n_2_S032_ICAed_raw.fif', 'n_2_S025_ICAed_raw.fif', 'n_2_S035_ICAed_raw.fif', 'n_2_S022_ICAed_raw.fif', 'n_2_S026_ICAed_raw.fif', 'n_2_S031_ICAed_raw.fif']
26


In [4]:
# Import the csv data
csv_data = DATA_ROOT / "Alice(EEG_mat_and stimuli)" / "AliceChapterOne-EEG.csv"  # self-made LMM data form


word_onset_LIST = []
with open(csv_data, "r", encoding="UTF-8") as f:
    fileDF = pd.read_csv(f, sep=",")
    #print(fileDF.columns)
    # word_onset = fileDF["onset"]
    # print(word_onset, type(word_onset))

    word_onset_essentials_DF = fileDF.iloc[:,[0, 1, 2] ]     # first column
    #print(word_onset_essentials_DF["onset"], word_onset_essentials_DF["onset"])
    #print()

    for i in range(1, 13):
        print(i)
        #w_S = word_onset_essentials_DF.loc[:, [i]["Word"]]
        #wOnset_F = word_onset_essentials_DF.loc[:, 2]
        #print(w_S)

        wOnset_DF = word_onset_essentials_DF.loc[word_onset_essentials_DF["Segment"] == 2, :]
        print(wOnset_DF, type(wOnset_DF))
        print(wOnset_DF["onset"])
        """
        if 
        w_S = word_onset_essentials_DF.iloc[i, 0]
        wOnset_F = word_onset_essentials_DF.iloc[i, 2]
        print(w_S, wOnset_F)
        """

1
         Word  Segment      onset
174       but        2   0.479840
175      when        2   0.592424
176       the        2   0.810806
177    Rabbit        2   0.912952
178  actually        2   1.415810
..        ...      ...        ...
346       was        2  58.681660
347     going        2  58.861252
348        to        2  59.148599
349    happen        2  59.251158
350      next        2  59.687374

[177 rows x 3 columns] <class 'pandas.core.frame.DataFrame'>
174     0.479840
175     0.592424
176     0.810806
177     0.912952
178     1.415810
         ...    
346    58.681660
347    58.861252
348    59.148599
349    59.251158
350    59.687374
Name: onset, Length: 177, dtype: float64
2
         Word  Segment      onset
174       but        2   0.479840
175      when        2   0.592424
176       the        2   0.810806
177    Rabbit        2   0.912952
178  actually        2   1.415810
..        ...      ...        ...
346       was        2  58.681660
347     going        2  58

In [22]:
for subject in ESL_SUBJECTS[0:3]:
    print("subject_num=", subject)
    
    # 1. Load Raw as an Eelbrain-compatible object
    raw = mne.io.read_raw_fif(EEG_DIR / f'{subject}', preload=True)
    raw_sfreq = raw.info['sfreq']
    
    # 2. Get the events for the 12 tapes
    events_DICT = eelbrain.load.mne.events(raw)
    #print(events_DICT["event"])
    
    """
    if len(events_DICT["event"])==12:
        pass
    else:
        print(subject[:3], "tape_length", )"""

subject_num= n_2_S030_ICAed_raw.fif
subject_num= n_2_S027_ICAed_raw.fif
subject_num= n_2_S023_ICAed_raw.fif


In [19]:
trial_indexes = [STIMULI.index(stimulus) for stimulus in events_DICT['event']]
trial_indexes = [STIMULI.index(stimulus) for stimulus in events['event'] if stimulus in STIMULI]  # type(trial_indexes)==LIST
for stimulus in events_DICT['event']:
    if stimulus in STIMULI:
        STIMULI.index(stimulus)
    else:
        pass
for i, stimulus_idx in enumerate(trial_indexes[1:]):
    print("i = ", i)
    print("stimulus_idx", stimulus_idx)

ValueError: '99' is not in list

In [27]:
for subject in ESL_SUBJECTS[0:3]:
    print("subject_num=", subject[4:8])
    
    # 1. Load Raw as an Eelbrain-compatible object
    raw = mne.io.read_raw_fif(EEG_DIR / f'{subject}', preload=True)
    raw_sfreq = raw.info['sfreq']

    # 2. Get the events for the 12 tapes
    events_DICT = eelbrain.load.mne.events(raw)
    #trial_indexes = [STIMULI.index(stimulus) for stimulus in events_DICT['event']]
    trial_indexes = [STIMULI.index(stimulus) for stimulus in events_DICT['event'] if stimulus in STIMULI]  # type(trial_indexes)==LIST

    print(events_DICT)

subject_num= S030
i_start   trigger   event
-------------------------
3130      1         1    
8931      16        99   
8982      9         51   
9002      5         2    
15103     16        99   
15154     10        52   
15174     6         3    
21575     16        99   
21646     7         4    
28647     16        99   
28718     8         5    
35419     16        99   
35470     11        55   
35490     12        6    
41891     16        99   
41962     13        7    
48263     16        99   
48334     14        8    
54135     16        99   
54206     15        9    
60007     16        99   
60078     2         10   
66279     16        99   
66350     3         11   
72051     16        99   
72122     4         12   
subject_num= S027
i_start   trigger   event
-------------------------
3498      1         1    
9299      19        99   
9350      9         51   
9370      5         2    
15471     19        99   
15542     6         3    
21943     19        99   
21

In [28]:
## (Use this!!!)Trying the old way (Sub order 1st then tape 2nd)##

# epochs parameters
# Define your epoch window
# We use a 1.5s window (including buffers for HHSA)
tmin, tmax = -0.3, 1.2 

for subject in ESL_SUBJECTS[0:3]:
    print("subject_num=", subject)
    
    # 1. Load Raw as an Eelbrain-compatible object
    raw = mne.io.read_raw_fif(EEG_DIR / f'{subject}', preload=True)
    raw_sfreq = raw.info['sfreq']
    
    # 2. Get the events for the 12 tapes
    events_DICT = eelbrain.load.mne.events(raw)
    trial_indexes = [STIMULI.index(stimulus) for stimulus in events_DICT['event'] if stimulus in STIMULI]  # type(trial_indexes)==LIST

    if len(events_DICT['event'])==12:
        # Get the actual word onset based on EEG triggers datapoints
        all_tapes_epochs_LIST = []
        for i, stimulus_idx in enumerate(trial_indexes):
            print("tape_num=", stimulus_idx+1)
            
            # Find the word onset time based on the segment sequence    
            wOnset_perTape_DF = word_onset_essentials_DF.loc[word_onset_essentials_DF["Segment"] == stimulus_idx+1, :] #.to_numpy()
            print("wOnset_perTape_DF=", wOnset_perTape_DF)
            #wOnset_time_ndarray = wOnset_perTape_DF["onset"].to_numpy() #*raw_sfreq
            wOnset_datapoints_ndarray = wOnset_perTape_DF["onset"].to_numpy()*raw_sfreq
            print("wOnset_datapoints_ndarray=", wOnset_datapoints_ndarray[0:15], type(wOnset_datapoints_ndarray))
    
            # Get the tape start time
            tape_start_datapoints_npINT64 = events_DICT[i]['i_start']
            #tape_start_time_npFLOAT64 = tape_start_datapoints_npINT64 / raw_sfreq
            print("tape_start_datapoints_npINT64=", tape_start_datapoints_npINT64)
            #print("tape_start_time_npFLOAT64=", tape_start_time_npFLOAT64, type(tape_start_time_npFLOAT64))
    
            # Get the actual word onset time by the triggers
            #absolute_onsets_time_ndarray = tape_start_time_npFLOAT64 + wOnset_time_ndarray
            #print("absolute_onsets_time_ndarray=", absolute_onsets_time_ndarray[0:15])
            abs_wOnsets_dta_ndarray = tape_start_datapoints_npINT64 + wOnset_datapoints_ndarray
            print("abs_wOnsets_dta_ndarray=", abs_wOnsets_dta_ndarray[0:15])
            
            # To exclude the decimal but leave the integer along, and turn FLOAT into INT
            abs_wOnsets_dta_ndarray = np.trunc(abs_wOnsets_dta_ndarray).astype(int)
            print("rounded_abs_wOnsets_dta_ndarray=", abs_wOnsets_dta_ndarray[0:15])
            
            # Make epochs
            # Create the empty (N, 3) event matrix based on wOnset per tape
            wOnset_events = len(abs_wOnsets_dta_ndarray)
            wOnset_perTape_events = np.zeros((wOnset_events, 3), dtype=int)
            # Fill the columns
            wOnset_perTape_events[:, 0] = abs_wOnsets_dta_ndarray  # Column 0: The sample indices
            wOnset_perTape_events[:, 2] = stimulus_idx+1           # Column 2: The event ID (e.g., 1)
            print(wOnset_perTape_events)
            
            #word_perTape_epochs = mne.epochs(raw, tmin=tmin, tmax=tmax, baseline=None, events=wOnset_perTape_events)
            tape_perTape_epochs = mne.Epochs(raw, events=wOnset_perTape_events, event_id=stimulus_idx+1, tmin=tmin, tmax=tmax, baseline=None, preload=True)
            print(tape_perTape_epochs)
            all_tapes_epochs_LIST.append(tape_perTape_epochs)
            wOnset_epochs = mne.concatenate_epochs(all_tapes_epochs_LIST)
        wOnset_epochs.save(wOnset_DIR / Path('%s_ESLs_wOnset_epochs_allTapes_raw.fif' %subject[4:8]), overwrite=True)
        print("subject_num=", subject[:4], "wOnset epoch saved.")
    
    else:  # For those who only start on the 2nd tape
        # Get the actual word onset based on EEG triggers datapoints
        all_tapes_epochs_LIST = []
        for i, stimulus_idx in enumerate(trial_indexes[1:]):
            print("tape_num=", stimulus_idx+1)
            
            # Find the word onset time based on the segment sequence    
            wOnset_perTape_DF = word_onset_essentials_DF.loc[word_onset_essentials_DF["Segment"] == stimulus_idx+1, :] #.to_numpy()
            print("wOnset_perTape_DF=", wOnset_perTape_DF)
            #wOnset_time_ndarray = wOnset_perTape_DF["onset"].to_numpy() #*raw_sfreq
            wOnset_datapoints_ndarray = wOnset_perTape_DF["onset"].to_numpy()*raw_sfreq
            print("wOnset_datapoints_ndarray=", wOnset_datapoints_ndarray[0:15], type(wOnset_datapoints_ndarray))
    
            # Get the tape start time
            tape_start_datapoints_npINT64 = events_DICT[i]['i_start']
            #tape_start_time_npFLOAT64 = tape_start_datapoints_npINT64 / raw_sfreq
            print("tape_start_datapoints_npINT64=", tape_start_datapoints_npINT64)
            #print("tape_start_time_npFLOAT64=", tape_start_time_npFLOAT64, type(tape_start_time_npFLOAT64))
    
            # Get the actual word onset time by the triggers
            #absolute_onsets_time_ndarray = tape_start_time_npFLOAT64 + wOnset_time_ndarray
            #print("absolute_onsets_time_ndarray=", absolute_onsets_time_ndarray[0:15])
            abs_wOnsets_dta_ndarray = tape_start_datapoints_npINT64 + wOnset_datapoints_ndarray
            print("abs_wOnsets_dta_ndarray=", abs_wOnsets_dta_ndarray[0:15])
            
            # To exclude the decimal but leave the integer along, and turn FLOAT into INT
            abs_wOnsets_dta_ndarray = np.trunc(abs_wOnsets_dta_ndarray).astype(int)
            print("rounded_abs_wOnsets_dta_ndarray=", abs_wOnsets_dta_ndarray[0:15])
            
            # Make epochs
            # Create the empty (N, 3) event matrix based on wOnset per tape
            wOnset_events = len(abs_wOnsets_dta_ndarray)
            wOnset_perTape_events = np.zeros((wOnset_events, 3), dtype=int)
            # Fill the columns
            wOnset_perTape_events[:, 0] = abs_wOnsets_dta_ndarray  # Column 0: The sample indices
            wOnset_perTape_events[:, 2] = stimulus_idx+1           # Column 2: The event ID (e.g., 1)
            print(wOnset_perTape_events)
            
            #word_perTape_epochs = mne.epochs(raw, tmin=tmin, tmax=tmax, baseline=None, events=wOnset_perTape_events)
            tape_perTape_epochs = mne.Epochs(raw, events=wOnset_perTape_events, event_id=stimulus_idx+1, tmin=tmin, tmax=tmax, baseline=None, preload=True)
            print(tape_perTape_epochs)
            all_tapes_epochs_LIST.append(tape_perTape_epochs)
            wOnset_epochs = mne.concatenate_epochs(all_tapes_epochs_LIST)
        wOnset_epochs.save(wOnset_DIR / Path('%s_ESLs_wOnset_epochs_11Tapes_raw.fif' %subject[4:8]), overwrite=True)
        print("subject_num=", subject[:4], "wOnset epoch saved.")
    

"""
    # 5. Combine all words from all tapes into one Dataset
    # This 'ds' will have a column for 'EEG' and can have a column for 'Subject'
    ds_all_words = eelbrain.combine(all_tapes_epochs)
    
    # Now you can access the data for HHSA:
    # eeg_data = ds_all_words['eeg'].get_data()
    """

subject_num= n_2_S030_ICAed_raw.fif
tape_num= 2
wOnset_perTape_DF=          Word  Segment      onset
174       but        2   0.479840
175      when        2   0.592424
176       the        2   0.810806
177    Rabbit        2   0.912952
178  actually        2   1.415810
..        ...      ...        ...
346       was        2  58.681660
347     going        2  58.861252
348        to        2  59.148599
349    happen        2  59.251158
350      next        2  59.687374

[177 rows x 3 columns]
wOnset_datapoints_ndarray= [ 47.9839572  59.2424402  81.0806402  91.2952402 141.5810402 191.8667402
 238.5605402 255.7777402 321.1728402 343.1803402 359.5218402 379.0771402
 438.4907402 563.7331402 574.7717402] <class 'numpy.ndarray'>
tape_start_datapoints_npINT64= 3130
abs_wOnsets_dta_ndarray= [3177.9839572 3189.2424402 3211.0806402 3221.2952402 3271.5810402
 3321.8667402 3368.5605402 3385.7777402 3451.1728402 3473.1803402
 3489.5218402 3509.0771402 3568.4907402 3693.7331402 3704.7717402]
rounde

/var/folders/dt/yj00f34n5x52_p314tnwfjkw0000gn/T/ipykernel_30538/1641304678.py:107: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  wOnset_epochs = mne.concatenate_epochs(all_tapes_epochs_LIST)
/var/folders/dt/yj00f34n5x52_p314tnwfjkw0000gn/T/ipykernel_30538/1641304678.py:107: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  wOnset_epochs = mne.concatenate_epochs(all_tapes_epochs_LIST)
/var/folders/dt/yj00f34n5x52_p314tnwfjkw0000gn/T/ipykernel_30538/1641304678.py:107: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  wOnset_epochs = mne.concatenate_epochs(all_tapes_epochs_LIST)
/var/folders/dt/yj00f34n5x52_p314tnwfjkw0000gn/T/ipykernel_30538/1641304678.py:107: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  wOnset_epochs 

tape_num= 4
wOnset_perTape_DF=           Word  Segment      onset
535       must        4   0.398386
536         be        4   0.689428
537    getting        4   0.840938
538  somewhere        4   1.126350
539       near        4   1.542892
..         ...      ...        ...
744      think        4  68.262788
745        you        4  68.437370
746      could        4  68.593016
747     manage        4  68.760635
748         it        4  69.129919

[214 rows x 3 columns]
wOnset_datapoints_ndarray= [ 39.8385787  68.9427631  84.0937631 112.6349631 154.2891631 177.2879631
 184.4716631 225.1791631 234.7573631 248.5197631 312.5805631 335.3288631
 347.3015631 380.8254631 399.0834631] <class 'numpy.ndarray'>
tape_start_datapoints_npINT64= 8982
abs_wOnsets_dta_ndarray= [9021.8385787 9050.9427631 9066.0937631 9094.6349631 9136.2891631
 9159.2879631 9166.4716631 9207.1791631 9216.7573631 9230.5197631
 9294.5805631 9317.3288631 9329.3015631 9362.8254631 9381.0834631]
rounded_abs_wOnsets_dta_ndarra

/var/folders/dt/yj00f34n5x52_p314tnwfjkw0000gn/T/ipykernel_30538/1641304678.py:107: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  wOnset_epochs = mne.concatenate_epochs(all_tapes_epochs_LIST)
/var/folders/dt/yj00f34n5x52_p314tnwfjkw0000gn/T/ipykernel_30538/1641304678.py:107: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  wOnset_epochs = mne.concatenate_epochs(all_tapes_epochs_LIST)
/var/folders/dt/yj00f34n5x52_p314tnwfjkw0000gn/T/ipykernel_30538/1641304678.py:107: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  wOnset_epochs = mne.concatenate_epochs(all_tapes_epochs_LIST)


tape_num= 7
wOnset_perTape_DF=           Word  Segment      onset
1139  Suddenly        7   0.586736
1140       she        7   1.179532
1141      came        7   1.418988
1142      upon        7   1.694362
1143         a        7   2.051148
...        ...      ...        ...
1312       her        7  61.175179
1313      head        7  61.294906
1314    though        7  61.675837
1315       the        7  61.845655
1316   doorway        7  61.965383

[178 rows x 3 columns]
wOnset_datapoints_ndarray= [ 58.6736359 117.9532195 141.8988195 169.4362195 205.1148195 230.4974195
 274.7968195 299.9396195 337.0553195 425.6539195 446.0076195 471.1505195
 479.5315195 527.4226195 650.7423195] <class 'numpy.ndarray'>
tape_start_datapoints_npINT64= 15154
abs_wOnsets_dta_ndarray= [15212.6736359 15271.9532195 15295.8988195 15323.4362195 15359.1148195
 15384.4974195 15428.7968195 15453.9396195 15491.0553195 15579.6539195
 15600.0076195 15625.1505195 15633.5315195 15681.4226195 15804.7423195]
rounded_abs_wO

...

/var/folders/dt/yj00f34n5x52_p314tnwfjkw0000gn/T/ipykernel_30538/1641304678.py:107: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  wOnset_epochs = mne.concatenate_epochs(all_tapes_epochs_LIST)
/var/folders/dt/yj00f34n5x52_p314tnwfjkw0000gn/T/ipykernel_30538/1641304678.py:107: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  wOnset_epochs = mne.concatenate_epochs(all_tapes_epochs_LIST)
/var/folders/dt/yj00f34n5x52_p314tnwfjkw0000gn/T/ipykernel_30538/1641304678.py:107: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  wOnset_epochs = mne.concatenate_epochs(all_tapes_epochs_LIST)


tape_num= 11
wOnset_perTape_DF=              Word  Segment      onset
1835          she       11   0.274485
1836        could       11   0.489783
1837          see       11   0.701863
1838           it       11   1.064477
1839        quite       11   1.279987
...           ...      ...        ...
1987           to       11  53.797174
1988         make       11  53.890245
1989          one       11  54.211688
1990  respectable       11  54.517325
1991       person       11  55.331149

[157 rows x 3 columns]
wOnset_datapoints_ndarray= [ 27.448538   48.9783265  70.1863265 106.4477265 127.9987265 161.5225265
 213.0055265 236.9511265 245.3320265 357.8762265 369.5823265 386.6109265
 420.1347265 433.3048265 489.5769265] <class 'numpy.ndarray'>
tape_start_datapoints_npINT64= 27027
abs_wOnsets_dta_ndarray= [27054.448538  27075.9783265 27097.1863265 27133.4477265 27154.9987265
 27188.5225265 27240.0055265 27263.9511265 27272.3320265 27384.8762265
 27396.5823265 27413.6109265 27447.1347265 27460.

/var/folders/dt/yj00f34n5x52_p314tnwfjkw0000gn/T/ipykernel_30538/1641304678.py:107: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  wOnset_epochs = mne.concatenate_epochs(all_tapes_epochs_LIST)
/var/folders/dt/yj00f34n5x52_p314tnwfjkw0000gn/T/ipykernel_30538/1641304678.py:108: RuntimeWarning: This filename (/Users/neuroling/Downloads/DINGHSIN_Results/Alice_Experiments_Results/EEG_ESLs/Alice_ESLs_wOnset_raw_epochs/S023_ESLs_wOnset_epochs_11Tapes_raw.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  wOnset_epochs.save(wOnset_DIR / Path('%s_ESLs_wOnset_epochs_11Tapes_raw.fif' %subject[4:8]), overwrite=True)


subject_num= n_2_ wOnset epoch saved.


"\n    # 5. Combine all words from all tapes into one Dataset\n    # This 'ds' will have a column for 'EEG' and can have a column for 'Subject'\n    ds_all_words = eelbrain.combine(all_tapes_epochs)\n    \n    # Now you can access the data for HHSA:\n    # eeg_data = ds_all_words['eeg'].get_data()\n    "

In [138]:
"""
## (Not using this version) Trying the old way (Tape order 1st then subject cut 2nd)##

# epochs parameters
# Define your epoch window
# We use a 1.5s window (including buffers for HHSA)
tmin, tmax = -0.3, 1.2 

# Get the actual word onset based on EEG triggers datapoints
for i, stimulus_idx in enumerate(trial_indexes):
    # Find the word onset time based on the segment sequence    
    wOnset_perTape_DF = word_onset_essentials_DF.loc[word_onset_essentials_DF["Segment"] == i+1, :] #.to_numpy()
    #wOnset_time_ndarray = wOnset_perTape_DF["onset"].to_numpy() #*raw_sfreq
    wOnset_datapoints_ndarray = wOnset_perTape_DF["onset"].to_numpy()*raw_sfreq
    print("wOnset_datapoints_ndarray=", wOnset_datapoints_ndarray[0:15], type(wOnset_datapoints_ndarray))

    for subject in SUBJECTS[0:3]:
        print("subject_num=", subject)
        print("tape_num=", i+1)
        
        # 1. Load Raw as an Eelbrain-compatible object
        raw = mne.io.read_raw_fif(EEG_DIR / f'{subject}', preload=True)
        raw_sfreq = raw.info['sfreq']
        
        # 2. Get the events for the 12 tapes
        events_DICT = eelbrain.load.mne.events(raw)

        # Get the tape start time
        tape_start_datapoints_npINT64 = events_DICT[i]['i_start']
        #tape_start_time_npFLOAT64 = tape_start_datapoints_npINT64 / raw_sfreq
        print("tape_start_datapoints_npINT64=", tape_start_datapoints_npINT64)
        #print("tape_start_time_npFLOAT64=", tape_start_time_npFLOAT64, type(tape_start_time_npFLOAT64))

        # Get the actual word onset time by the triggers
        #absolute_onsets_time_ndarray = tape_start_time_npFLOAT64 + wOnset_time_ndarray
        #print("absolute_onsets_time_ndarray=", absolute_onsets_time_ndarray[0:15])
        abs_wOnsets_dta_ndarray = tape_start_datapoints_npINT64 + wOnset_datapoints_ndarray
        print("abs_wOnsets_dta_ndarray=", abs_wOnsets_dta_ndarray[0:15])
        
        # To exclude the decimal but leave the integer along, and turn FLOAT into INT
        abs_wOnsets_dta_ndarray = np.trunc(abs_wOnsets_dta_ndarray).astype(int)
        print("rounded_abs_wOnsets_dta_ndarray=", abs_wOnsets_dta_ndarray[0:15])

        # Make epochs
        # Create the empty (N, 3) event matrix based on wOnset per tape
        wOnset_events = len(abs_wOnsets_dta_ndarray)
        wOnset_perTape_events = np.zeros((wOnset_events, 3), dtype=int)
        # Fill the columns
        wOnset_perTape_events[:, 0] = abs_wOnsets_dta_ndarray  # Column 0: The sample indices
        wOnset_perTape_events[:, 2] = i+1           # Column 2: The event ID (e.g., 1)
        print(wOnset_perTape_events)

        
        #word_perTape_epochs = mne.epochs(raw, tmin=tmin, tmax=tmax, baseline=None, events=wOnset_perTape_events)
        tape_perTape_epochs = mne.Epochs(raw, events=wOnset_perTape_events, event_id=i+1, tmin=tmin, tmax=tmax, baseline=None, preload=True)
        print(tape_perTape_epochs)
    all_tapes_epochs.append(tape_word_epochs)
"""        
    

    """(Gemini-Produced)
    for i, stimulus_idx in enumerate(trial_indexes):
        # We find the start of the tape in the raw EEG
        tape_start = events[i]['i_start']#['time']
        print([i], tape_start, type(tape_start))
        
        # Get word onset times for this specific tape from your list
        # word_onsets[stimulus_idx] is your impulse predictor
        onsets = word_onsets[stimulus_idx].time.times[word_onsets[stimulus_idx].x > 0]
        print(onsets, len(onsets))
    
        
        # Convert relative word times to absolute EEG times
        absolute_onsets = tape_start + onsets
        
        # Create segments (Epochs) directly in Eelbrain
        # This is much faster and more accurate than manual padding/cropping
        tape_word_epochs = mne.epochs(
            raw, tmin=tmin, tmax=tmax, baseline=None, 
            events=absolute_onsets
        )
        all_tapes_epochs.append(tape_word_epochs)
    
    # 5. Combine all words from all tapes into one Dataset
    # This 'ds' will have a column for 'EEG' and can have a column for 'Subject'
    ds_all_words = eelbrain.combine(all_tapes_epochs)
    
    # Now you can access the data for HHSA:
    # eeg_data = ds_all_words['eeg'].get_data()
    """

wOnset_datapoints_ndarray= [  4.6     56.2721  78.4543 125.6929 135.2925 161.6327 231.0749 279.9918
 293.8712 330.449  348.4082 365.4596 410.6667 427.4286 439.8799] <class 'numpy.ndarray'>
subject_num= S44_Alice-natives_sfreq-100_raw.fif
tape_num= 1
tape_start_datapoints_npINT64= 172
abs_wOnsets_dta_ndarray= [176.6    228.2721 250.4543 297.6929 307.2925 333.6327 403.0749 451.9918
 465.8712 502.449  520.4082 537.4596 582.6667 599.4286 611.8799]
rounded_abs_wOnsets_dta_ndarray= [176 228 250 297 307 333 403 451 465 502 520 537 582 599 611]
[[ 176    0    1]
 [ 228    0    1]
 [ 250    0    1]
 [ 297    0    1]
 [ 307    0    1]
 [ 333    0    1]
 [ 403    0    1]
 [ 451    0    1]
 [ 465    0    1]
 [ 502    0    1]
 [ 520    0    1]
 [ 537    0    1]
 [ 582    0    1]
 [ 599    0    1]
 [ 611    0    1]
 [ 650    0    1]
 [ 662    0    1]
 [ 675    0    1]
 [ 715    0    1]
 [ 761    0    1]
 [ 777    0    1]
 [ 886    0    1]
 [ 910    0    1]
 [ 921    0    1]
 [ 967    0    1]
 [ 998 

NameError: name 'tape_word_epochs' is not defined

In [11]:
# Load stimuli
# ------------
# Make sure to name the stimuli so that the TRFs can later be distinguished
# Load the gammatone-spectrograms; use the time axis of these as reference
gammatone = [eelbrain.load.unpickle(PREDICTOR_audio_DIR / f'{stimulus}~gammatone-8.pickle') for stimulus in STIMULI]

# Resample the spectrograms to 100 Hz (time-step = 0.01 s), which we will use for TRFs
gammatone = [x.bin(0.01, dim='time', label='start') for x in gammatone]

# Pad onset with 100 ms and offset with 1 second; make sure to give the predictor a unique name as that will make it easier to identify the TRF later
gammatone = [eelbrain.pad(x, tstart=-0.100, tstop=x.time.tstop + 1, name='gammatone') for x in gammatone]

# Load the broad-band envelope and process it in the same way
envelope = [eelbrain.load.unpickle(PREDICTOR_audio_DIR / f'{stimulus}~gammatone-1.pickle') for stimulus in STIMULI]  # Load in the data
envelope = [x.bin(0.01, dim='time', label='start') for x in envelope]
envelope = [eelbrain.pad(x, tstart=-0.100, tstop=x.time.tstop + 1, name='envelope') for x in envelope]
onset_envelope = [eelbrain.load.unpickle(PREDICTOR_audio_DIR / f'{stimulus}~gammatone-on-1.pickle') for stimulus in STIMULI]
onset_envelope = [x.bin(0.01, dim='time', label='start') for x in onset_envelope]
onset_envelope = [eelbrain.pad(x, tstart=-0.100, tstop=x.time.tstop + 1, name='onset') for x in onset_envelope]

# Load onset spectrograms and make sure the time dimension is equal to the gammatone spectrograms
gammatone_onsets = [eelbrain.load.unpickle(PREDICTOR_audio_DIR / f'{stimulus}~gammatone-on-8.pickle') for stimulus in STIMULI]
gammatone_onsets = [x.bin(0.01, dim='time', label='start') for x in gammatone_onsets]
gammatone_onsets = [eelbrain.set_time(x, gt.time, name='gammatone_on') for x, gt in zip(gammatone_onsets, gammatone)]

In [8]:
## Got to the wrong length of the word onset

## Gemini-Produced
## To segment the raw EEG based on word start time == word onset time ##
# ... your loading code ...

for subject in SUBJECTS:
    # 1. Load Raw as an Eelbrain-compatible object
    raw = mne.io.read_raw_fif(EEG_DIR / f'{subject}', preload=True)
    raw_sfreq = raw.info['sfreq']
    
    # 2. Get the events for the 12 tapes
    events = eelbrain.load.mne.events(raw)
    trial_indexes = [STIMULI.index(stimulus) for stimulus in events['event']]
    
    # 3. Define your epoch window
    # We use a 1.5s window (including buffers for HHSA)
    tmin, tmax = -0.3, 1.2 
    
    # 4. Extract segments for each tape
    # This creates a list of NDVars, each containing the epochs for one tape
    all_tapes_epochs = []
    
    for i, stimulus_idx in enumerate(trial_indexes):
        # We find the start of the tape in the raw EEG
        tape_start = events[i]['i_start']#['time']
        
        # Get word onset times for this specific tape from your list
        # word_onsets[stimulus_idx] is your impulse predictor
        onsets = word_onsets[stimulus_idx].time.times[word_onsets[stimulus_idx].x > 0]
        print(onsets, len(onsets))

        """
        # Convert relative word times to absolute EEG times
        absolute_onsets = tape_start + onsets
        
        # Create segments (Epochs) directly in Eelbrain
        # This is much faster and more accurate than manual padding/cropping
        tape_word_epochs = mne.epochs(
            raw, tmin=tmin, tmax=tmax, baseline=None, 
            events=absolute_onsets
        )
        all_tapes_epochs.append(tape_word_epochs)

    # 5. Combine all words from all tapes into one Dataset
    # This 'ds' will have a column for 'EEG' and can have a column for 'Subject'
    ds_all_words = eelbrain.combine(all_tapes_epochs)
    
    # Now you can access the data for HHSA:
    # eeg_data = ds_all_words['eeg'].get_data()
    """

NameError: name 'word_onsets' is not defined

In [22]:
## This is the time point of every word onset time according to the Alice csv file ##

# Load word tables and convert tables into continuous time-series with matching time dimension
word_tables = [eelbrain.load.unpickle(PREDICTOR_word_DIR / f'{stimulus}~word.pickle') for stimulus in STIMULI]
word_onsets = [eelbrain.event_impulse_predictor(gt.time, ds=ds, name='word') for gt, ds in zip(gammatone, word_tables)]

#print(word_tables[-1]) #  the onset & offset of each tape
print("===== WORD ONSET IS DOWN BELOW =====")
print(word_onsets[0][0:5])
# This is the original Durations of 12 tapes based on gammatone
# Extract the duration of the stimuli, so we can later match the EEG to the stimuli
durations = [gt.time.tmax for stimulus, gt in zip(STIMULI, gammatone)]


"""
# Estimate TRFs
# -------------
# Loop through subjects to estimate TRFs
for subject in SUBJECTS:
    subject_trf_dir = TRF_DIR / subject[:3]
    subject_trf_dir.mkdir(exist_ok=True)
    # Generate all TRF paths so we can check whether any new TRFs need to be estimated
    trf_paths = {model: subject_trf_dir / f'{subject[:3]} {model}.pickle' for model in models}
    # Skip this subject if all files already exist
    #if all(path.exists() for path in trf_paths.values()):
        #continue
    # Load the EEG data
    raw = mne.io.read_raw_fif(EEG_DIR / f'{subject}', preload=True)  # subject /
    # Band-pass filter the raw data between 0.5 and 20 Hz
    raw.filter(0.5, 20)
    # Interpolate bad channels
    raw.interpolate_bads()
    # Extract the events marking the stimulus presentation from the EEG file
    events = eelbrain.load.fiff.events(raw)
    # Not all subjects have all trials; determine which stimuli are present
    trial_indexes = [STIMULI.index(stimulus) for stimulus in events['event']]
    # Extract the EEG data segments corresponding to the stimuli
    trial_durations = [durations[i] for i in trial_indexes]
    eeg = eelbrain.load.fiff.variable_length_epochs(events, -0.100, trial_durations, connectivity='auto')  #, decim=5 #decim=5 meaning to resample to sfreq=100Hz
    # Since trials are of unequal length, we will concatenate them for the TRF estimation.
    eeg_concatenated = eelbrain.concatenate(eeg)
    
    pprint(models.items)
    """

===== WORD ONSET IS DOWN BELOW =====
<NDVar 'word': 500 time>


"\n# Estimate TRFs\n# -------------\n# Loop through subjects to estimate TRFs\nfor subject in SUBJECTS:\n    subject_trf_dir = TRF_DIR / subject[:3]\n    subject_trf_dir.mkdir(exist_ok=True)\n    # Generate all TRF paths so we can check whether any new TRFs need to be estimated\n    trf_paths = {model: subject_trf_dir / f'{subject[:3]} {model}.pickle' for model in models}\n    # Skip this subject if all files already exist\n    #if all(path.exists() for path in trf_paths.values()):\n        #continue\n    # Load the EEG data\n    raw = mne.io.read_raw_fif(EEG_DIR / f'{subject}', preload=True)  # subject /\n    # Band-pass filter the raw data between 0.5 and 20 Hz\n    raw.filter(0.5, 20)\n    # Interpolate bad channels\n    raw.interpolate_bads()\n    # Extract the events marking the stimulus presentation from the EEG file\n    events = eelbrain.load.fiff.events(raw)\n    # Not all subjects have all trials; determine which stimuli are present\n    trial_indexes = [STIMULI.index(stimulu

In [ ]:
"""
import numpy as np
import matplotlib.pyplot as plt
import eelbrain
from pathlib import Path
import re
import emd  # Ensure you ran: pip install EMD-signal
from scipy.signal import hilbert
"""

## Conduct HHSA ##
## Step one


# ==========================================
# 1. CORE ANALYSIS FUNCTIONS
# ==========================================
def get_inst_freq_amp(signal, fs):
    """
    Calculates instantaneous amplitude and frequency using Hilbert Transform.
    """
    analytic_signal = hilbert(signal)
    amplitude = np.abs(analytic_signal)
    instantaneous_phase = np.unwrap(np.angle(analytic_signal))
    
    # Derivative of phase for frequency
    inst_freq = np.diff(instantaneous_phase) / (2.0 * np.pi) * fs
    # Pad last point to match length
    inst_freq = np.append(inst_freq, inst_freq[-1])
    
    return amplitude, inst_freq

def run_hhsa(signal, fs):
    """
    Performs Two-Layer EMD with Mirror Padding for short signals.
    """
    if signal.ndim > 1: signal = signal.flatten()
    
    n_samples = len(signal)
    
    # Safety Check for empty/tiny signals
    if n_samples < 10: return None

    # --- A. MIRROR PADDING (Crucial for 114-point TRFs) ---
    # We reflect the signal to make it 3x longer so EMD works
    pad_width = n_samples
    padded_signal = np.pad(signal, pad_width, mode='reflect')
    
    # --- B. LAYER 1: CARRIER DECOMPOSITION ---
    try:
        # Use emd.sift.sift (Quinn library)
        # max_imfs=5 is enough for a simple TRF response
        imfs_layer1 = emd.sift.sift(padded_signal, max_imfs=5)
        imfs_layer1 = imfs_layer1.T  # Transpose to (N_IMFs, N_Time)
    except Exception:
        return None
        
    holo_points = []
    
    for imf_c in imfs_layer1:
        # Un-pad: Extract the middle 'real' part
        imf_c_real = imf_c[pad_width : pad_width + n_samples]
        
        # Get Carrier Frequency & Envelope
        env_c, freq_c = get_inst_freq_amp(imf_c_real, fs)
        
        # Skip flat/empty IMFs
        if np.sum(np.abs(env_c)) < 1e-10: continue

        # --- C. LAYER 2: AM DECOMPOSITION ---
        # Pad the envelope before sifting
        padded_env = np.pad(env_c, pad_width, mode='reflect')
        
        try:
            imfs_layer2 = emd.sift.sift(padded_env, max_imfs=5)
            imfs_layer2 = imfs_layer2.T
        except:
            continue
            
        for imf_am in imfs_layer2:
            # Un-pad AM result
            imf_am_real = imf_am[pad_width : pad_width + n_samples]
            
            _, freq_am = get_inst_freq_amp(imf_am_real, fs)
            power_am = imf_am_real**2
            
            # Filter for valid graph range
            mask = (freq_c > 0) & (freq_c < fs/2) & (freq_am > 0) & (freq_am < fs/2)
            idx = np.where(mask)[0]
            
            if len(idx) > 0:
                # Store [Carrier_Freq, AM_Freq, Power]
                points = np.vstack((freq_c[idx], freq_am[idx], power_am[idx])).T
                holo_points.append(points)
                
    if not holo_points: return None
    return np.vstack(holo_points)

# ==========================================
# 2. EXPERIMENT CONFIGURATION
# ==========================================
DATA_ROOT = Path("/Users/neuroling/Downloads/DINGHSIN_Results/Alice_Experiments_Results")
EEG_DIR = DATA_ROOT / 'EEG_ESLs' / 'Alice_ESL_ICAed_fif'
TRF_DIR = DATA_ROOT / 'TRFs_ESLs'

# Extract Subjects
ESL_SUBJECTS = [path.name for path in EEG_DIR.iterdir() if re.match(r'n_2_S\d*', path.name)]
print(f"Found {len(ESL_SUBJECTS)} subjects.")

# Settings
TARGET_MODEL = 'Fzero'
LIMIT_CARRIER = 5  # Hz
LIMIT_AM = 5       # Hz
NBINS = 50

group_spectrum_sum = None
n_subjects_processed = 0

# ==========================================
# 3. MAIN LOOP
# ==========================================
print(f"Starting Group HHSA on Model: {TARGET_MODEL}")

for subject in ESL_SUBJECTS:
    subject_id_short = subject[4:8] # 'S010'
    file_path = TRF_DIR / subject_id_short / f'{subject_id_short} {TARGET_MODEL}.pickle'
    
    if not file_path.exists():
        continue

    try:
        # --- LOAD DATA ---
        trf_obj = eelbrain.load.unpickle(file_path)
        
        # Handle h vs h_scaled
        if hasattr(trf_obj, 'h_scaled'):
            data_ndvar = trf_obj.h_scaled
        else:
            data_ndvar = trf_obj.h

        # Handle Tuple (Partitioned Data)
        if isinstance(data_ndvar, tuple):
            data_ndvar = data_ndvar[0]

        # --- EXTRACT PREDICTOR ---
        trf_final = None
        
        # Strategy 1: Try name 'envelope'
        try:
            trf_final = data_ndvar['Fzero']
        except:
            # Strategy 2: Try Index
            # If dims are [predictor, time], we grab the 2nd predictor (index 1)
            # Assuming the order is [F0, Envelope]
            dims = data_ndvar.dimnames
            non_time_dims = [d for d in dims if d != 'time' and d != 'sensor']
            
            if len(non_time_dims) > 0:
                dim_name = non_time_dims[0]
                # Index 1 = Envelope. Change to 0 if you want F0.
                trf_final = data_ndvar.sub(**{dim_name: 1}) 
            else:
                trf_final = data_ndvar

        # Average Sensors
        if 'sensor' in trf_final.dimnames:
            trf_final = trf_final.mean('sensor')
            
        trf_vector = trf_final.x
        fs = 1.0 / trf_final.time.tstep
        
        # --- RUN HHSA ---
        # print(f"  Processing {subject_id_short}...")
        holo_data = run_hhsa(trf_vector, fs)
        
        if holo_data is None: 
            continue
            
        # --- DEBUG PRINT ---
        # Print the average frequencies found for this subject
        avg_fc = np.mean(holo_data[:, 0])
        avg_fam = np.mean(holo_data[:, 1])
        print(f"  Subject {subject_id_short}: Avg Carrier={avg_fc:.2f}Hz, Avg AM={avg_fam:.2f}Hz")
        # -------------------
        

        # --- BIN RESULTS ---
        fc = holo_data[:, 0]
        fam = holo_data[:, 1]
        power = holo_data[:, 2]
        
        H_subj, xedges, yedges = np.histogram2d(fc, fam, bins=NBINS, weights=power,
                                           range=[[0, LIMIT_CARRIER], [0, LIMIT_AM]])
        
        if group_spectrum_sum is None:
            group_spectrum_sum = H_subj
        else:
            group_spectrum_sum += H_subj
            
        n_subjects_processed += 1
        
    except Exception as e:
        print(f"  [Error] Failed on {subject_id_short}: {e}")
        continue

# ==========================================
# 4. PLOT FINAL RESULT
# ==========================================
if n_subjects_processed > 0:
    group_avg_spectrum = group_spectrum_sum / n_subjects_processed
    
    plt.figure(figsize=(10, 8))
    # Note: .T is used because imshow expects [rows, cols] = [y, x]
    plt.imshow(group_avg_spectrum.T, origin='lower', cmap='jet', aspect='auto',
               extent=[0, LIMIT_CARRIER, 0, LIMIT_AM], interpolation='gaussian')
    
    plt.colorbar(label='Modulation Power (a.u.)')
    plt.xlabel('Carrier Frequency (Hz)')
    plt.ylabel('AM Frequency (Hz)')
    plt.title(f'Group HHSA (N={n_subjects_processed})\nModel: {TARGET_MODEL} | Predictor: Envelope')
    
    # Diagonal line (1:1 coupling)
    plt.plot([0, LIMIT_AM], [0, LIMIT_AM], 'w--', alpha=0.5)
    plt.savefig(TRF_DIR / 'ESLs_Fzero_HHSA_TRF.png')
    plt.show()
    print("Success!")
else:
    print("No subjects processed. Please check if pickle files contain the expected data structure.")

In [ ]:
"""
import numpy as np
import matplotlib.pyplot as plt
import eelbrain
from pathlib import Path
import emd
from scipy.signal import hilbert
"""
# Version 1 of IMFs in each layers (=second EMD)

# ==========================================
# 1. CONFIGURATION
# ==========================================
DATA_ROOT = Path("/Users/neuroling/Downloads/DINGHSIN_Results/Alice_Experiments_Results")
TRF_DIR = DATA_ROOT / 'TRFs_ESLs'
TARGET_MODEL = 'Fzero+envelope'
TARGET_SUBJECT = 'S010'  # <--- Change this to look at different subjects

# Which Layer 1 IMF do you want to decompose further?
# 0 = First IMF (Fastest/Highest Freq), 1 = Second IMF, etc.
TARGET_IMF_INDEX = 1 

# ==========================================
# 2. LOAD DATA (Robust Loading)
# ==========================================
file_path = TRF_DIR / TARGET_SUBJECT / f'{TARGET_SUBJECT} {TARGET_MODEL}.pickle'
print(f"Loading {TARGET_SUBJECT}...")

try:
    trf_obj = eelbrain.load.unpickle(file_path)
    
    # Handle h vs h_scaled
    if hasattr(trf_obj, 'h_scaled'):
        data_ndvar = trf_obj.h_scaled
    else:
        data_ndvar = trf_obj.h
        
    # Handle Tuple
    if isinstance(data_ndvar, tuple):
        data_ndvar = data_ndvar[0]

    # Extract Predictor (Strategy: Index)
    # Assumes order is [F0, Envelope] -> Index 1 is Envelope
    # If your model is just 'envelope', it might be Index 0.
    try:
        # Try finding 'envelope' by name
        trf_final = data_ndvar['envelope']
        pred_name = "Envelope"
    except:
        # Fallback to Index 1
        dims = data_ndvar.dimnames
        non_time_dims = [d for d in dims if d != 'time' and d != 'sensor']
        if len(non_time_dims) > 0:
            trf_final = data_ndvar.sub(**{non_time_dims[0]: 1})
            pred_name = "Predictor (Index 1)"
        else:
            trf_final = data_ndvar
            pred_name = "Predictor"

    # Average Sensors
    if 'sensor' in trf_final.dimnames:
        trf_final = trf_final.mean('sensor')
        
    signal = trf_final.x
    times = trf_final.time.times
    fs = 1.0 / trf_final.time.tstep

    if signal.ndim > 1: signal = signal.flatten()
    print(f"Data Loaded: {len(signal)} samples")

except Exception as e:
    print(f"Error loading: {e}")
    exit()

# ==========================================
# 3. PROCESSING (With Mirror Padding)
# ==========================================

# --- A. Layer 1 Decomposition ---
pad_width = len(signal)
padded_signal = np.pad(signal, pad_width, mode='reflect')

# Run EMD (Sift)
imfs_layer1_padded = emd.sift.sift(padded_signal, max_imfs=5)

# Un-pad Layer 1
# Note: emd output is (Samples, IMFs)
imfs_layer1 = imfs_layer1_padded[pad_width : pad_width + len(signal), :]
n_imfs1 = imfs_layer1.shape[1]

# --- B. Layer 2 Decomposition (Target IMF) ---
# Extract the target IMF (e.g., IMF 0)
target_imf_padded = imfs_layer1_padded[:, TARGET_IMF_INDEX]

# Get Envelope (using Hilbert)
analytic = hilbert(target_imf_padded)
envelope_padded = np.abs(analytic)

# Run EMD on Envelope
imfs_layer2_padded = emd.sift.sift(envelope_padded, max_imfs=4)

# Un-pad Layer 2
envelope_real = envelope_padded[pad_width : pad_width + len(signal)]
imfs_layer2 = imfs_layer2_padded[pad_width : pad_width + len(signal), :]
n_imfs2 = imfs_layer2.shape[1]

# ==========================================
# 4. PLOTTING
# ==========================================

# --- FIGURE 1: LAYER 1 (Carrier) ---
fig1, axes1 = plt.subplots(n_imfs1 + 1, 1, figsize=(10, 8), sharex=True)
fig1.suptitle(f"Layer 1: Carrier Decomposition ({TARGET_SUBJECT})", fontsize=14)

# Plot Original Signal
axes1[0].plot(times, signal, 'k', label='Original TRF')
axes1[0].set_title(f"Original Signal: {pred_name}")
axes1[0].legend(loc='upper right')

# Plot IMFs
for i in range(n_imfs1):
    ax = axes1[i + 1]
    ax.plot(times, imfs_layer1[:, i], 'b')
    ax.set_ylabel(f"IMF {i+1}")
    ax.grid(True, alpha=0.3)

axes1[-1].set_xlabel("Time (s)")
plt.tight_layout()
#plt.savefig(TRF_DIR / 'ESLs_S010_layer1-IMFs_HHSA_TRF.png')
plt.show()


# --- FIGURE 2: LAYER 2 (Amplitude Modulation) ---
fig2, axes2 = plt.subplots(n_imfs2 + 1, 1, figsize=(10, 8), sharex=True)
fig2.suptitle(f"Layer 2: AM Decomposition of Carrier IMF {TARGET_IMF_INDEX+1}", fontsize=14)

# Plot Envelope
axes2[0].plot(times, envelope_real, 'r', label=f'Envelope of IMF {TARGET_IMF_INDEX+1}')
axes2[0].set_title("Amplitude Envelope (Input to Layer 2)")
axes2[0].legend(loc='upper right')

# Plot AM IMFs
for i in range(n_imfs2):
    ax = axes2[i + 1]
    ax.plot(times, imfs_layer2[:, i], 'g')
    ax.set_ylabel(f"AM IMF {i+1}")
    ax.grid(True, alpha=0.3)

axes2[-1].set_xlabel("Time (s)")
plt.tight_layout()
plt.savefig(TRF_DIR / 'ESLs_S010_layer2-IMF2_HHSA_TRF.png')
plt.show()